In [ ]:
import openai
from diffusers import DiffusionPipeline
import torch
from torchvision.utils import save_image
from PIL import Image
import matplotlib.pyplot as plt
openai.api_key = 'YOUR_OPEN_API_KEY'

def test_ChatGPT(question):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": question}]
    )
    response = completion.choices[0].message.content
    return response
 
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipeline.to("cuda")

words=[]
cnt=0
initial = input("ChatGPTの返答の表示:ON…1")
# 対話の開始
while True:
    # ユーザーからの入力を受け取る
    user_input = input("しりとりのお題,語:")
    
    err=(
        'これは[ん]で終わる単語です。' if user_input[-1]==('ん' or 'ン') else 
        'この単語は既に使われています。' if user_input in words else
        '')
        
    if err:
        print(err+'あなたの負けです')
        break
    if user_input.lower() == "降参":
        print("私の勝ちですね。ありがとうございました。")
        break
    if user_input.lower() == "終了":
        print("しりとりを終了します。")
        break 
    words.append(user_input)    
    if cnt ==0:
        question =  "しりとりをしましょう。お題は"+ user_input +"です。一語だけ返してください。"
    else:
        question = "しりとりの次の語は"+ user_input + "です。一語だけ返してください。"
    cnt+=1  
    if(initial==1):
        print(question)
        
    print(question)
    # 回答の表示
    output = test_ChatGPT(question)
    print(output)
    er=(
        'これは[ん]で終わる単語です。' if output[-1]==('ん' or 'ン') else''
        'この単語は既に使われています。' if output in words else''
        
    )
        
        
    if er:
        print(er+'私の負けです')
        break        
    words.append(output) 
    
    question1=output +"を英訳してください。一語だけ返してください。"
    print(question1)
    # 回答の表示
    output1 = test_ChatGPT(question1)
    print(output1) 
    g_image=pipeline(output1).images[0] 
    # 画像の保存
    g_image.save(f'g_image.png')

# 保存した画像の表示
    image = Image.open('g_image.png')
    plt.imshow(image)
    plt.axis('off')
    plt.show()